# Generation for NLP Baseline Code

```
python3.10 -m venv --system-site-packages /data/ephemeral/home/py310
source /data/ephemeral/home/py310/bin/activate
pip install --upgrade pip
```
위에 커맨드를 사용하여 가상환경을 만들고 IDE의 커널을 생성한 가상환경으로 변경해주세요.

## Install Packages

In [ ]:
# !pip install --no-cache-dir torch==2.9.1+cu128 --index-url https://download.pytorch.org/whl/cu128
# !pip install --no-cache-dir -r requirements.txt

## Import Necessary Libraries

In [ ]:
import torch
import transformers
from ast import literal_eval
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import AutoPeftModelForCausalLM, LoraConfig

import os
from optimum.neuron import NeuronSFTTrainer, NeuronSFTConfig

pd.set_option('display.max_columns', None)


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) # magic number :)

### Load Data

In [ ]:
# Load the train dataset
# TODO Train Data 경로 입력
dataset = pd.read_csv('train.csv')

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)

# Convert to DataFrame
df = pd.DataFrame(records)
dataset = Dataset.from_pandas(df)


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
PROMPT_NO_QUESTION_PLUS = """지문:
{paragraph}

질문:
{question}

선택지:
{choices}

1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:"""

PROMPT_QUESTION_PLUS = """지문:
{paragraph}

질문:
{question}

<보기>
{question_plus}

선택지:
{choices}

1, 2, 3, 4, 5 중에 하나를 정답으로 고르세요.
정답:"""

In [ ]:

def process_data(example):
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(example["choices"])])

    if example["question_plus"]:
        user_message = PROMPT_QUESTION_PLUS.format(
            paragraph=example["paragraph"],
            question=example["question"],
            question_plus=example["question_plus"],
            choices=choices_string,
        )
    else:
        user_message = PROMPT_NO_QUESTION_PLUS.format(
            paragraph=example["paragraph"],
            question=example["question"],
            choices=choices_string,
        )

    # Qwen 등의 Chat 모델을 위한 메시지 포맷 구성
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant. 지문을 읽고 질문의 답을 구하세요."}, # System Prompt 추가
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": f"{example['answer']}"}
    ]

    return {"messages": messages}

processed_dataset = dataset.map(process_data)

In [ ]:
processed_dataset = []
for i in range(len(dataset)):
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])

    # <보기>가 있을 때
    if dataset[i]["question_plus"]:
        user_message = PROMPT_QUESTION_PLUS.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            choices=choices_string,
        )
    # <보기>가 없을 때
    else:
        user_message = PROMPT_NO_QUESTION_PLUS.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            choices=choices_string,
        )

    # chat message 형식으로 변환
    processed_dataset.append(
        {
            "id": dataset[i]["id"],
            "messages": [
                {"role": "system", "content": "지문을 읽고 질문의 답을 구하세요."},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": f"{dataset[i]['answer']}"}
            ],
            "label": dataset[i]["answer"],
        }
    )

processed_dataset = Dataset.from_pandas(pd.DataFrame(processed_dataset))
processed_dataset

In [ ]:
# 본인의 Huggingface auth token 입력
## Jupyter lab에서 로그인 하는 textbox가 나오지 않을 경우, terminal에서 로그인 하실 수 있습니다.
# !huggingface-cli login

In [ ]:
model_id = "Qwen/Qwen3-8B"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.padding_side = 'right' # Neuron 학습 시 right padding 권장

if tokenizer.chat_template is None:
    tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    low_cpu_mem_usage=True
)


peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_args = NeuronSFTConfig(
    output_dir="outputs_qwen3_neuron",
    max_seq_length=1024,      # 입력 길이 (데이터셋 분포에 따라 2048 등으로 늘릴 수 있음)
    num_train_epochs=3,
    per_device_train_batch_size=1, # 8B 모델은 메모리를 더 차지하므로 1로 유지하거나 AWS 인스턴스 사양에 맞춰 조정
    gradient_accumulation_steps=8, # 배치가 작을 경우 그래디언트 누적 사용 권장
    learning_rate=1e-5,       # 모델이 커지면 학습률을 조금 낮추는 것이 일반적
    logging_steps=1,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=2,
    bf16=True,
    use_cache=False,
    report_to="none",
)

processed_dataset = processed_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = processed_dataset['train']
eval_dataset = processed_dataset['test']

In [ ]:
trainer = NeuronSFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
)

In [ ]:
print(f"Model: {model_id}")
print("Neuron 컴파일 및 학습이 시작됩니다. 초기 컴파일에 몇 분(10분 이상) 소요될 수 있습니다...")
trainer.train()

# 9. 모델 저장
trainer.save_model("outputs_qwen3_neuron_final")